<center>

<h1 style="text-align:center"> (Side) Effects </h1>
</center>

## Why Side Effects

* We have only used **purely functional** feature of OCaml
* Our study of lambda calculus used only **purely functional** features

* Not quite true!
  + We have used `print_endline`, `printf` and other features to display our results to screen.
  + We have used `fresh` function to implement substitution in Lambda Calculus.
* It is sometimes useful to write programs that have **side effects**

## Side effects

Side effects include

* Mutating (i.e., destructively updating) the values of program state. 
* Reading from standard input, printing to standard output.
* Reading and writing to files, sockets, pipes etc.
* ...
* Composing, sending and receiving emails, editing documents, writing this slide, etc.

## Side effects in OCaml

* OCaml programs can include side effects
* Features
  + Mutations: Reference cells, Arrays, Mutable record fields
  + I/O of all sorts
* In this lecture, **Mutations**

$
\require{color}
\newcommand{\cpurple}[1]{{\color{\purple}{\text{#1}}}}
\newcommand{\cred}[1]{{\color{\red}{\text{#1}}}}
$

## Reference cells

* Aka "refs" or "ref cell"
* Essentially a pointer to a location in memory
* The binding of a variable to a ref cell is $\cpurple{immutable}$ 
  + but the contents of the ref cell may $\cred{change}$.

## Reference cells

Allocate new ref cells with `ref`

In [ ]:
let r = ref 1

Update using `:=` and read using `!`.

In [ ]:
r := !r + 1

In [ ]:
!r

## Reference Cells: Types

In [ ]:
ref

In [ ]:
(!)

In [ ]:
(:=)

## Sequencing using `;`

We can chain together expressions with side effects using the semi-colon operator

In [ ]:
r := !r + 1;
!r

This is equivalent to

In [ ]:
let _ = r := !r + 1 in
!r

## Implementing a counter

In [ ]:
let make_counter init =
  let c = ref init in
  fun () -> 
    (c := !c + 1; !c)

In [ ]:
let next = make_counter 0

In [ ]:
next ()

In [ ]:
next ()

Aside: A function of the form `fun () -> ...` is also called a **thunk**.

## Side effects make reasoning hard

* **Referential transparency** allows replacing $e$ with $v$ if $e =_{\beta} v$.
* Side effects break referential transparency.

## Referential transparency

Consider the function `foo`:

In [ ]:
let foo x = x + 1

In [ ]:
let baz = foo 10

`baz` can now be optimised to

In [ ]:
let baz = 11

## Referential transparency

Consider the function `bar`:

In [ ]:
let next = make_counter 0
let bar x = x + next()

In [ ]:
let qux = bar 10

Can we now optimise `qux` to:

In [ ]:
let qux = 11

**NO!** Referential transparency breaks under side effects. 

## Example: Implementing pointers using `ref`

Desired properties:
* Points to a location whose value can be changed
* Can be `null`

In [ ]:
type 'a pointer = 'a option ref

In [ ]:
let malloc (v : 'a) : 'a pointer = ref (Some v)

In [ ]:
let p = malloc 42

## Example: Implementing pointers using `ref`

In [ ]:
exception Segfault

let deref (p : 'a pointer) : 'a = 
  match !p with
  | None -> raise Segfault
  | Some v -> v

In [ ]:
deref p

In [ ]:
deref (ref None)

In [ ]:
let assign (p : 'a pointer) (v: 'a option) : unit = 
  p := v

In [ ]:
assign p (Some 2);
deref p

## Example 2: Recursive functions without `rec` keyword

In [ ]:
let rec fact n = if n = 0 then 1 else n * fact (n-1)

In [ ]:
fact 5

Let's implement `fact` without using `rec` but with function references!

## Example 2: Recursive functions without `rec` keyword

Step 1: Declare a function reference to a dummy function. Need to ensure that the type matches the function to be implemented.

In [ ]:
let fact0 = ref (fun x -> x + 0)

Step 2: Redefine the original function without `rec` keyword, replacing the recursive call with the function reference.

In [ ]:
let fact_norec n = if n = 0 then 1 else n * !fact0 (n-1)

## Example 2: Recursive functions without `rec` keyword

Step 3: Assign the function reference to the newly defined function

In [ ]:
fact0 := fact_norec

In [ ]:
fact_norec 5

Also called "tying the recursive knot".

## Aliases

References may create aliases.

What is the result of this program?

In [ ]:
let x = ref 10 in
let y = ref 10 in
let z = x in 
z := !z + 1;
!x + !y

<center>
    
<img src="heap2.svg" width="350">
</center>

* Variables `z` and `x` are said to be **aliases**
  + They refer to the same object in the program heap.

## Equality

* The `=` that we have been using is known as **structural equality**
  + Checks whether the values are structurally equal. 
  + `[1;2;3] = [1;2;3]` evaluates to `true`. 
* Because of references, one may also want to ask whether two expressions are **aliases**
  + This equality is known as **physical equality**.
  + OCaml uses `==` to check for physical equality.

## Equality : Types

In [ ]:
(=)

In [ ]:
(==)

## Equality : Quiz
```ocaml
let l1 = [1;2;3];; 
let l2 = l1;; 
let l3 = [1;2;3];;
let r1 = ref l1;;
let r2 = r1;;
let r3 = ref l3;;
```
which of the following are true?

(1)`l1 = l2` (2)`l1 == l2` (3) `l1 = l3` (4) `l1 == l3`  (5) `l1 = [1;2;3]`

(6) `l1 == [1;2;3]` (7) `r1 == r2`  (8) `r1 = r2` (9) `r1 == r3` (10) `r1 = r3`

## Equality : Quiz

In [ ]:
let l1 = [1;2;3];; 
let l2 = l1;; 
let l3 = [1;2;3];;
let r1 = ref l1;;
let r2 = r1;;
let r3 = ref l3;;

## Equality

Which of the following are true?

(1)`l1 = l2` (2)`l1 == l2` (3) `l1 = l3` (4) `l1 == l3`  (5) `l1 = [1;2;3]`

(6) `l1 == [1;2;3]` (7) `r1 == r2`  (8) `r1 = r2` (9) `r1 == r3` (10) `r1 = r3`

References are structurally equal iff their contents are structurally equal.

In [ ]:
l1 == l2

## Mutable Record Fields

Ref cells are essentially syntactic sugar over **mutable record fields**.

```ocaml
type 'a ref = { mutable contents: 'a }
let ref x = { contents = x }
let ( ! ) r = r.contents
let ( := ) r newval = r.contents <- newval
```

* This type is declared in `Pervasives`


## Doubly-linked list

In [ ]:
(* The type of elements *)
type 'a node = { 
  v : 'a;
  mutable next : 'a node option;
  mutable prev : 'a node option
}

(* The type of list *)
type 'a dllist = 'a node option ref

## Double-linked list

In [ ]:
let create () : 'a dllist = ref None
let first (t : 'a dllist) = !t
let is_empty (t : 'a dllist) = !t = None

let v node = node.v
let next node = node.next
let prev node = node.prev

## Doubly-linked list

In [ ]:
(* [insert_first t v] inserts a new node [n] with value [v] 
   as the first node in the list [t]. Returns [n]. *)
let insert_first (t : 'a dllist) v =
  let n = { prev = None; next = !t; v = v} in
  begin match !t with
  | Some old_first -> old_first.prev <- Some n
  | None -> ()
  end;
  t := Some n;
  n

## Doubly-linked list

In [ ]:
(* [insert_after n v] inserts a new node [n'] whose value 
   is [v] after the node [n]. Returns [n']. *)
let insert_after n v =
  let n' = { v = v; prev = Some n; next = n.next } in
  begin match n.next with
  | Some old_next -> old_next.prev <- Some n'
  | None -> ()
  end;
  n.next <- Some n';
  n'

## Doubly-linked list

In [ ]:
(* [remove t n] removes the node [n] from the list [t] *)
let remove (t:'a dllist) n =
  begin match n.prev with
  | Some prev -> prev.next <- n.next
  | None -> t := n.next
  end;
  begin match n.next with
  | Some next -> next.prev <- n.prev;
  | None -> ()
  end;
  n.prev <- None;
  n.next <- None

## Doubly-linked list

In [ ]:
(* [iter t f] applies [f] on each value in the list 
   from left to right *)
let iter (t : 'a dllist) (f : 'a -> unit) =
  let rec loop l = match l with
    | None -> ()
    | Some el -> f (v el); loop (next el)
  in
  loop !t

## Doubly-linked list

In [ ]:
let l = create ();;
let n0 = insert_first l 0;;
let n1 = insert_after n0 1;;
insert_after n1 2

## Doubly-linked list

In [ ]:
iter l (Printf.printf "%d\n%!")

## Arrays

Collection type with efficient random access.

In [ ]:
let a = [| 1;2;3 |]

Array elements are retrieved using `a.(i)` syntax where `a` is the array and `i` is the index.

In [ ]:
a.(0)

Arrays bounds are checked at access.

In [ ]:
a.(3)

## Arrays

Array can be updated with `<-` syntax.

In [ ]:
a.(1) <- 0; 
a

In [ ]:
a.(1)

* Arrays behave very similar to arrays in C
 + See [Array module](https://caml.inria.fr/pub/docs/manual-ocaml/libref/Array.html) documentation in the OCaml standard library

## Benefits of immutability

* Programmer doesn’t have to think about aliasing
    + can concentrate on other aspects of code
* Language implementation is free to share objects on the heap, which is cheap
* Often easier to reason about whether code is correct
* Perfect fit for concurrent programming

But
* Some data structures (hash tables, arrays, ...) are more efficient if imperative

Recommendation

* Use immutable data structures unless performance can't be compromised